In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from PIL import Image
import numpy as np
import ast
from modeci_mdf.execution_engine import EvaluableGraph
from modeci_mdf.interfaces.pytorch import pytorch_to_mdf


In [ ]:
def image_loader(image_path):
    # Define transformation pipeline
    loader = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor()
    ])
    # Open image using PIL
    image = Image.open(image_path)
    # Apply transformation pipeline to image
    image = loader(image).float()
    # Convert image tensor to float data type
    image = image.float()
    # Add batch dimension to the tensor by unsqueezing it along the 0th dimension
    image = image.unsqueeze(0)
    # Return preprocessed image tensor
    return image


In [ ]:


# Load a pre-trained ResNet18 model
model = models.resnet18(pretrained=True)

# Set the model to evaluation mode
model.eval()

# Define an input image path
input_image_path = "test_img.png"

# Load the input image
input_image = image_loader(input_image_path)

# Get the model output and predicted label
output = model(input_image)
_, pred_label = torch.max(output.data, 1)

# Print the predicted label
print("Predicted Label:", pred_label.item())

# Convert the PyTorch model to MDF format
graph = pytorch_to_mdf(model)
print(graph)

# Define the input value for the model
input_values = {"input.1": input_image.numpy()}

# Create an EvaluableGraph object from the MDF graph
eval_graph = EvaluableGraph.from_mdf(graph)

# Evaluate the model on the input data
results = eval_graph.evaluate(input_values=input_values)

# Print the model output
print("Model Output:", results["output"])
